In [8]:
#Installing FinRL
# %%capture
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git
  
# #Installing Optuna
# !pip install optuna
# !pip install dm_tree
!pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [9]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import optuna
%matplotlib inline
from finrl.apps import config
from optuna.integration import PyTorchLightningPruningCallback
from finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.neo_finrl.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.neo_finrl.env_stock_trading.env_stocktrading_np import StockTradingEnv as StockTradingEnv_numpy
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.drl_agents.rllib.models import DRLAgent as DRLAgent_rllib
from finrl.neo_finrl.data_processor import DataProcessor
import joblib
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import ray
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [10]:
%%capture
## Collecting and preprocessing data
#Custom ticker list dataframe download
ticker_list = config.DOW_30_TICKER
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-10-01',
                     ticker_list = ticker_list).fetch_data()
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [31]:
## Setup datasets

# View a sample dataframe
# processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

# Split into train and trade datasets
# train = data_split(processed_full, '2009-01-01','2020-07-01')
# trade = data_split(processed_full, '2020-07-01','2021-10-31')

train = data_split(processed_full, '2019-01-01','2020-12-31')
trade = data_split(processed_full, '2021-01-01','2021-10-31')
print(f"Training data size {len(train)}")
print(f"Trading data size {len(trade)}")
print ("Training data sample")
train.head()
print ("Trading data sample")
trade.head()

stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Training data size 14616
Trading data size 5423
Training data sample


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,38.326294,148158800.0,2.0,-2.007526,44.298934,35.280062,37.867376,-91.583074,42.250808,41.034358,46.272397,23.219999,43.730072
0,2019-01-02,AMGN,192.520004,193.199997,188.949997,175.788910,3009100.0,2.0,-1.306191,186.820077,163.947086,49.241492,-30.974754,4.899123,177.207583,177.384066,23.219999,43.730072
0,2019-01-02,AXP,93.910004,96.269997,93.769997,91.575745,4175400.0,2.0,-3.386529,109.349546,83.747229,41.204986,-97.772283,26.709417,99.213765,99.777659,23.219999,43.730072
0,2019-01-02,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,2.0,-5.550588,339.116371,287.252028,47.010001,-21.712387,13.611972,314.427161,331.956153,23.219999,43.730072
0,2019-01-02,CAT,124.029999,127.879997,123.000000,116.861320,4783200.0,2.0,-0.672706,123.794204,107.837728,48.229091,-5.208495,0.873482,116.370004,118.540167,23.219999,43.730072


Trading data sample


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-01-04,AAPL,133.520004,133.610001,126.760002,128.617111,143301900.0,0.0,3.441190,136.578796,117.958036,56.613043,64.551755,8.436326,124.117176,120.070758,26.969999,37.684545
0,2021-01-04,AMGN,231.250000,231.250000,223.669998,219.858322,3088200.0,0.0,-0.412324,225.892903,215.124502,48.078722,28.665878,1.082054,219.309955,221.072275,26.969999,37.684545
0,2021-01-04,AXP,121.300003,121.800003,116.849998,116.679779,3472100.0,0.0,0.921035,123.386761,112.696052,54.746515,-8.552931,3.618340,117.849149,110.284337,26.969999,37.684545
0,2021-01-04,BA,210.000000,210.199997,202.490005,202.720001,21225600.0,0.0,1.293101,241.661280,205.335718,50.209209,-132.990300,12.708438,220.805333,194.563834,26.969999,37.684545
0,2021-01-04,CAT,183.000000,185.979996,180.250000,178.422684,4078300.0,0.0,2.036097,179.122987,172.662123,59.272566,141.727560,34.923782,174.135918,167.630383,26.969999,37.684545


Stock Dimension: 29, State Space: 291


In [32]:
%%capture
## Setup Training environment
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()
agent = DRLAgent(env = env_train)

# SAC_PARAMS = {
#     "batch_size": 128,
#     "buffer_size": 1000000,
#     "learning_rate": 0.0001,
#     "learning_starts": 100,
#     "ent_coef": "auto_0.1",
# }

# model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

# trained_sac = agent.train_model(model=model_sac, 
#                              tb_log_name='sac',
#                              total_timesteps=60000)

model_ddpg = agent.get_model("ddpg")



In [ ]:
%%time
%%capture
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

CPU times: user 3.09 ms, sys: 0 ns, total: 3.09 ms
Wall time: 2.99 ms
